# Statistical Data Management Session 1: Visualisation of Qualitative Data

## Introduction to JupyterLab

1. Run the following cell of code. Look up the shortcut to run a cell via the Run menu.

In [ ]:
print("Hello World!")

2. Delete the cell. Add it again. Delete it again using a shortcut, which you can find in the Edit menu.

3. Click the Enable Debugger button next to the Python 3 (ipykernel) message. Set breakpoints in the code below and run it step by step by using the buttons in the CALLSTACK subwindow. Fix the bug and disable the breakpoints again. Use code-completion with tab key.

In [ ]:
a = 5
b = "10"
print(a+b)
print(a*b)

4. Run the following cell and then delete the cell.

In [ ]:
c = 10

5. Now run the following cell.

In [ ]:
print(c)

6. You get an output, despite the fact that the code defining the variable `c` has been deleted! This is because JupyterLab runs code cell-by-cell, not top to bottom. Click the round button on top that says *Restart the kernel* and try running the cell again. It is wise to restart the kernel if you deleted code, so you don't rely on variables that have been defined by no-longer existing code.


## Reading Data from a file

The packages we will be using are:
* pandas: a panda *dataframe* is the main data structure we will use
* numpy: for calculations
* matplotlib.pyplot: for plotting

The line `%matplotlib inline` is to make plots appear below the cell.

Never change these imports and their aliases.

Run the following cell. The file is in the *shared* folder.

This data set contains data on televised campaign advertising broadcast by a major presidential candidate in Peru from the 1980s through 2011 (with the exception of 1995 and 2000 elections). Major candidates are defined as the first- and second-place finishers, plus any third candidate who received over 20% of the valid vote. (Boas, Taylor. 2015. "Presidential campaign advertising in Chile, Brazil, and Peru". Qualitative Data Repository. https://doi.org/10.5064/F62Z13FQ. QDR Main Collection. V1)

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

conn = sqlite3.connect("../../shared/boas_peru_metadata.db")
df_raw = pd.read_sql_query("SELECT * FROM videos_peru", conn)
df_raw

We will not use this complete data set, but will count how many videos each candidate has made and work with this data frame. Later we will see how we can perform this count with an SQL query, but for now we simply give you the query. Add a print to see the result. Use `df[<column_name>]` to access a certain column.

In [ ]:
df = pd.read_sql_query("SELECT candidate, COUNT(*) as count FROM videos_peru GROUP BY candidate", conn)
df

You can add a column by simply defining `df[<new_column_name>] = <some calculation on a different column>`. Give the newly defined column below a meaningful name.

In [ ]:
df['relative_frequency'] = df['count']/df['count'].sum()
print(df)

## Bar Plot

Look at the documentation of the bar plot method: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.bar.html. Two parameters are mandatory: `x` (categories to be plotted on the horizontal axis) and `height` (respective heights of the categories). Others are optional, recognisable by the fact that they are given a default value, which you can optionally overrule with a custom value. Once all options and plots have been defined, end with `plt.show()` and `plt.close()`, which makes the plot.

In [ ]:
plt.bar(df['candidate'], df['count'])
plt.show()
plt.close()

What did the following cell do?

Now order candidates in descending order of number of videos. Look up the syntax online.

In [ ]:
df = df.sort_values('candidate')
df = df.sort_values('count', ascending=False)
df

Run the following cell. You are *not* required to be able to do this yourself, and are not claiming this is an esthetically optimal plot, but we want to give you an idea of what is possible with this powerful library.

In [ ]:
plt.figure(figsize=(10,6))
plt.xlabel('Candidate', fontsize=18)
plt.ylabel('Number of videos', fontsize=18)
plt.title('Presidential campaign advertising in Chile, Brazil, and Peru', fontsize=18)
plt.tick_params(labelsize=16, labelcolor='b')
plt.xticks(rotation=45)
plt.bar(df['candidate'],df['count'], color=(0.9,0.6,0.1), width=0.5, edgecolor='g', linewidth=3)
plt.show()
plt.close()

## Pie Charts

Not all data are interesting for our research. What does the following cell achieve?

In [ ]:
df_filtered = df[df['count'] >= 10]
df_filtered = df_filtered.reset_index()
df_filtered

Make a pie chart of these filtered data, starting from the documentation: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.pie.html. Note that only the wedge sizes are mandatory, but it's a good idea to provide respective labels, to see which wedge corresponds to which candidate.

In [ ]:
plt.pie(df_filtered['count'], labels=df_filtered['candidate'])
plt.show()
plt.close()

Run the following cell of code. Toy around with the parameters and variables to see what they mean or do (or look it up in the matplotlib API). Again, you are not required to be able to reproduce this plot.

In [ ]:
plt.figure(figsize=(6,6))

names_to_be_exploded = ["Vargas Llosa","Keiko"]
explosion =[0]*df_filtered.shape[0]
indices_to_be_exploded = df_filtered[df_filtered['candidate'].isin(names_to_be_exploded)].index.values
for i in indices_to_be_exploded:
    explosion[i] = 0.2
    
total = df_filtered['count'].sum()
colors = ['gold','olivedrab','salmon','chocolate','cadetblue','indigo','lightskyblue']
# full list at: https://matplotlib.org/stable/gallery/color/named_colors.html
patches,texts,autotexts = plt.pie(df_filtered['count'], labels=df_filtered['candidate'], colors=colors, explode=explosion, startangle=40, autopct = lambda p: '{:.0f}'.format(p * total / 100))
plt.setp(autotexts, color='white')
plt.setp(texts, size=14)
threshold = 30

for label in autotexts:
    if int(label.get_text())>threshold:
        label.set_fontsize(18)
        label.set_fontweight('bold')

plt.legend(title='Candidates',loc='right', bbox_to_anchor = [1.7,0.5])

plt.show()
plt.close()